In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/test.csv /content
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/train.csv /content

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/antecedents.tsv /content
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/consequents.tsv /content

!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/outputs.antecedents/submit_results.csv /content/antecedent_results.csv
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/outputs.consequents/submit_results.csv /content/consequent_results.csv

In [ ]:
import pandas as pd

prefix = '/content/'
test_df = pd.read_csv(prefix + 'test.csv', header=None) # change to test.csv
test_df=test_df.drop(index=0)

test_df = pd.DataFrame({
    'id':test_df[0],
    'text': test_df[1]
})
# display 
test_df.head()

In [ ]:
import pandas as pd

prefix = '/content/'

a_df = pd.read_csv(prefix + 'antecedents.tsv', sep='\t', delimiter=None, header=None)
a_df = pd.DataFrame({
    'id': a_df[0],
    'labels':a_df[1],
    'text': a_df[2],
    'start':a_df[3],
    'end': a_df[4]
})
#a_df.sort_values(by=['id','text'], ascending=[1,0])

c_df = pd.read_csv(prefix + 'consequents.tsv', sep='\t', delimiter=None, header=None)
c_df = pd.DataFrame({
    'id': c_df[0],
    'labels':c_df[1],
    'text': c_df[2],
    'start':c_df[3],
    'end': c_df[4]
})

rc_df = pd.read_csv(prefix + 'consequent_results.csv', header=None)
rc_df = pd.DataFrame({
    'id': rc_df[0],
    'labels':rc_df[1]
})
rc_df.sort_values(by=['labels'], ascending=False, inplace = True)

ra_df = pd.read_csv(prefix + 'antecedent_results.csv', header=None)
ra_df = pd.DataFrame({
    'id': ra_df[0],
    'labels':ra_df[1]
})
ra_df.sort_values(by=['labels'], ascending=False, inplace = True)
ra_df.head(10)

In [ ]:
pip install allennlp allennlp-models

In [ ]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.structured_prediction
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/biaffine-dependency-parser-ptb-2020.04.06.tar.gz")

original = 'If that was my daughter, I would have asked If I did something wrong.'    
prediction = predictor.predict(
  sentence = original
)
prediction

In [ ]:
def extract_dependents(prediction):
 root = None
 text = str()

 for k, v in prediction.items():
  if (k=='hierplane_tree'):
    for key, val in v.items():
      if key=='root':
        root = val
      if key=='text': 
        text = val   

 root.update({'text': text})
 return extract_children(root,'','advmod advcl nsubj ccomp xcomp pcomp pobj dobj iobj') 

In [ ]:
def extract_children(obj, include='', exclude=''):
  arr = list()
  args = list()
  spans = list()
  frames = list()

  def extract(obj, arr): 
    explore = False
    limit = 5 
    count = 0
    text = str()

    if isinstance(obj, dict):
      for k, v in obj.items():
        if include =='' and k=='attributes' and 'VERB' in v:
          if obj not in frames:
            frames.append(obj)

        if k=='nodeType':
          arr.append(v)

        if k=='text' and len(v)>0 and len(text)==0:  
          text = v

        if k=='children' and isinstance(v, list):
          if count < limit:
            count = count + 1
            extract(v, arr)   
             
    elif isinstance(obj, list):
      for item in obj:
        if isinstance(item, dict):
    
          for key, value in item.items():
            if key == 'nodeType':
              arr.append(value)
              if (include=='' or value in include) and (exclude=='' or not [v for v in arr if v in exclude]):
                #print('['+include+']', exclude,'=>', item['word'],'<=',arr)
                if item not in args: 
                  args.append(item) # frame argument must be found amoung children objects
                explore = True 
              else:  
                explore = False

            if key=='attributes' and 'CCONJ' in value:
              explore = False     

            if explore and key =='spans':
              if value not in spans:
                spans.append(value)  
            
          if explore and count < limit:
            count = count + 1
            extract(item, arr)

        arr.clear() 
    return text

  text = extract(obj, arr)
  return text, spans, args, frames

In [ ]:
def position(span):
  return span[0]['start']

In [ ]:
import re
regex = '\\\'\s+([^\']+)\s+\\\''
def compact(fragment):
# specify the number of replacements by changing the 4th argument
  result = re.sub(regex, '\''+r'\1'+'\'' , fragment)
  return result

In [ ]:
def normalize(fragment):
  r = fragment.strip()
  r = compact(r)
  r = r.replace('   ',' ') 
  r = r.replace('  ',' ')
  r = r.replace(' ,',',') 
  r = r.replace(' .','.')
  r = r.replace('Mr.','Mr. ')
  r = r.replace('Ms.','Ms. ')
  r = r.replace('Mrs.','Mrs. ')
  r = r.replace('.  ','. ') # Mr./Ms./Mrs.
  r = r.replace('... ','...')
  r = r.replace(' --- ','---')
  r = r.replace(' -- ','--')
  r = r.replace(' - ','-')
  r = r.replace(' !','!')
  r = r.replace(' ?','?')
  r = r.replace('$ ','$')
  r = r.replace('# ','#')
  r = r.replace(' %','%')
  r = r.replace('( ','(')
  r = r.replace(' )',')')
  r = r.replace(' ;',';')
  r = r.replace(' :',':') 
  r = r.replace(': ',':') #50/50 - carefull!
  r = r.replace(' nt','nt')
  r = r.replace(' (k)','(k)') #401(k)
  r = r.replace(' (tm)','(tm)')
  r = r.replace('n na ','nna ')
  r = r.replace(' \'m ','\'m ')
  r = r.replace('s \' ','s\' ') # was 'neurtralized'?
  r = r.replace(' \'s ','\'s ') # a 'surgical' strike!
  r = r.replace(' \'d ','\'d ')
  r = r.replace(' \'re ','\'re ')
  r = r.replace(' \'ve ','\'ve ')
  r = r.replace(' \'ll ','\'ll ')
  r = r.replace(' n\'t','n\'t') # what about UPPER()?
  r = r.replace(' \' ',' \'')
  r = r.rstrip()
  return r

In [ ]:
def explore(frame, condition, complement, sentence, removed, include='',exclude='', astart=-1, aend=-1, cstart=-1, cend=-1):
  deps = list()
  loops= list()
  if len(condition)==0:
    return deps, loops

  start = min(astart,cstart)
  end = max(aend,cend)
  spantext = sentence[start:end] # full (a-c) span(text can be 1 char off b/c sentence == original - " " removed )
  if len(spantext) <= 1:
    return deps, loops

  if spantext[-1] in ',;:.?!': # remove any punctuation that could trigger the *if not* logic below
    spantext = spantext[:-1]
 
  if astart > cend: 
    if condition.split(' ')[0] in 'had should': # 'higher than the 50% that would have been expected |*had* the technique not been used'
      spantext = str()
    if complement.split(' ')[-1] in 'although': # 'I should have tried to talk him out of it as *though* | my life *had* depended on it.'
      spantext = sentence[cstart:cend-10]       # 10 == len('as though')
  elif cstart > aend: 
    if condition.split(' ')[-1] in 'although':  # since *though* in 'although', above ends^____________ will *not* loops.append(spantext)
      spantext = sentence[astart:aend-10] 
    if complement.split(' ')[0] in 'had should':# 'Had he refused... , | he would have *had* to settle...' - will not unify (unfortunately!)
      spantext = str() 

  if not (' even ' in spantext.lower() or # '...as such they could have trouble | buying *even* with a ridiculous amount of mone'- don't unify
          ' if ' in spantext.lower() or   # 'If that was my daughter | would have asked *If* I did something wrong' - will not unify (sadly)
          ',' in spantext.lower() or
          ';' in spantext.lower() or
          ':' in spantext.lower() or 
          '.' in spantext.lower() or 
          '?' in spantext.lower() or 
          '!' in spantext.lower()
    ) and len(spantext) > 0:  
    if ((0 < cstart and cstart <= aend + 2 and astart < cstart) or 
        (0 < cstart and astart <= cend + 2 and cstart < astart) or        # 'they had already said *what* | I would have said' - will unify
        (0 < cstart and cstart == aend + 2 and condition.split(' ')[-1]  in 'although instead of i you we she he they that this these those which what whom who\'s when and why') or 
        (0 < cstart and astart == cend + 2 and complement.split(' ')[-1] in 'although instead of i you we she he they that this these those which what whom who\'s when and why')
    ):
      if spantext not in loops: # adjunct subspan
        #print(spantext, cstart, cend, complement)
        loops.append(spantext)

  text, spans, args, frames = extract_children(frame, include, exclude)
  if len(spans) == 0:
    return deps, loops

  for arg in args:    
    #print('?\"'+arg['word']+'\":',astart,aend,condition) 
    for span in sorted(spans,key=position): # all spans
      start=span[0]['start']
      end = span[0]['end']
      for r in removed:
        if r[0] <= start:
          start=start - 1
        if r[0] <= end:
          end = end - 1 
      
      spantext = sentence[start:end].strip() # dependent subspan of (a-)condition
      #print(spantext,start,end)
      if len(spantext) > 0 and len(spantext.split(' ')) == 1: # only a single word
        if astart <= start and end <= aend: 
          if spantext in condition and spantext not in deps:
            #print(spantext, start, end, condition)
            deps.append(spantext)     
            
    return deps, loops       

In [ ]:
def track(frames, antecedent, consequent, sentence, removed, astart, aend, cstart, cend):
  #print(astart, aend, cstart, cend)
  adeps = set()
  aloops= set()
  cdeps = set()
  cloops= set()
  if len(frames)==0:
    print('no frames detected!')
    return adeps, cdeps, aloops, cloops

  for frame in frames:
    #print('frame:', frame['word'])
    deps, loops = explore(frame, antecedent, consequent, sentence, removed, 'dep','advmod advcl nsubj ccomp xcomp pcomp pobj dobj iobj', astart, aend, cstart, cend)
    adeps.update(deps)
    aloops.update(loops)
    #deps, loops = explore(frame, consequent, antecedent, sentence, removed, 'advcl','',cstart,cend,astart,aend)
    #cdeps.update(deps)
    #cloops.update(loops)
    if len(adeps) > 0 or len(cdeps) > 0:
      return adeps, cdeps, aloops, cloops    
    adeps = set()
    cdeps = set()  

  return adeps, cdeps, aloops, cloops   

In [ ]:
def separate_from(md, pa):
  index = -1
  if pa in md and len(md) > len(pa):
    index = md.find(pa)
    if index > 0 :
      md = md[0:index]
      #print('|'+md+'{...}')
    elif index == 0:
      md = md.replace(pa,'')
      #print('|{...}'+md)  
  return md, index       

In [ ]:
def analyze(id, df, r_df, original, sentence, verbs=[], cstart=-1, cend=-1, first=True, override=False):
  print('________________________________________________________________________________________________________________________')
  apred = r_df.loc[r_df.id.str.contains(id)]
  if len(apred)==0:
    return id, 0, -1, -1, 0

  ctext = original[cstart:cend+1] # text playing consequent role
  print('|' if first else '|*','Fix','^=' if override else '=',ctext, cstart, cend)
 
  # check based on rules first = rlab 
  alab ='0' # assume antecedent false
  count= 0  # counter of predictions
  for p in apred.index: # [p] = index
    count = count + 1
    amid = apred['id'][p]  
    alab = apred['labels'][p] #predicted

    arule = df.loc[df.id.str.contains(amid)]
    assert len(arule) > 0

    for a in arule.index: # [a] = index 
      ralab = arule['labels'][a] 
      astart = arule['start'][a]
      aend = arule['end'][a]
      #atext = arule['text'][a]
      atext = original[astart:aend+1]
      
      print('|', ralab,'(',alab,')', atext, astart, aend) # atext is antecedent
      if first and override:
        return amid, 0, cstart, cend, 0 # pass through and return (c-)argument!!

      if (cstart <= astart and aend <= cend) or (astart <= cstart and cstart <= aend):
        if (override or alab==0) and len(apred.index) > count:
          print('x')
          continue  
        return amid, alab, min(astart,cstart), max(aend,cend) ,'-25'  

      adeps,cdeps,aloops,cloops = track(verbs, atext, ctext, sentence, removed, astart, aend, cstart, cend)
      print('| Res =',adeps,cdeps,aloops,cloops) # eg. 'set() set() set() set()'

      if ralab == 0:       
        if alab == 0 and not first and len(apred.index) > count:
          print('x')
          continue  

        if len(adeps) > 0 or len(aloops) > 0 or len(cdeps) or len(cloops) > 0:

          
          if not first and alab == 0 and atext not in ctext and ctext not in atext:
            if cstart > aend:
              aend = cend
              print('|   +=', sentence[astart:aend+1])
              return amid, alab, astart, aend,'-50'
            elif astart > cend:
              astart = cstart
              print('|   +=', sentence[astart:aend+1])
              return amid, alab, astart, aend,'-50'   

        if len(adeps) > 0 and len(cdeps) > 0:  
          return amid, alab, astart, aend,'-100'
        elif len(adeps) > 0 or len(cdeps) > 0:
          return amid, alab, astart, aend,'-75'
        elif len(aloops)==1 or len(cloops)==1:
          return amid, alab, min(astart,cstart), max(aend,cend) ,'-25'
              
      if ralab== 1:
        return amid, alab, astart, aend, 100
      elif alab==1:
        return amid, alab, astart, aend, 75 
      else: 
        return amid, alab, astart, aend, 50
      print('v')
  return amid, alab, astart, aend, 25

In [ ]:
def record(original):
  base=[(i, c) for i, c in enumerate(original)]
  return base

In [ ]:
def contract(text, base):
  removed = []
  for i in range(len(base)-1):
    sequence = base[i][1]
    if i < len(base) - 1 :
      sequence = sequence + base[i+1][1]
    if text.find(sequence,base[i][0]-i)==-1:
      removed.append(base[i])
  return removed

In [ ]:
original = 'Age may have dimmed their employment prospects, but older people often had the financial firepower to start up their own businesses - though should they have chosen to embark upon a second career, it would have been good doing something that they always wanted to do.'

prediction = predictor.predict(sentence = original.replace('\"',''))
text, spans, args, frames = extract_dependents(prediction)
#print(frames)
print(text)

antecedent = 'though should they have chosen to embark upon a second career'
astart=134  
aend = 194
consequent = 'would have been good doing something that they always wanted' 
cstart=200      
cend = 259

base = record(text)
sentence = normalize(text)
print(sentence)
removed = contract(sentence, base)
#print(removed)

adeps, cdeps, aloops, cloops = track(frames, antecedent, consequent, sentence, removed, astart, aend, cstart, cend)
print(adeps,cdeps)
print(aloops,cloops)

In [ ]:
import pandas as pd

from tqdm import tqdm
from time import sleep

submit_df = pd.DataFrame(columns=['sentenceID','antecedent_startid','antecedent_endid','consequent_startid','consequent_endid'])
report_df = pd.DataFrame(columns=['sentenceID','clabel','cprob','cdep','consequent','cstart','cend','antecedent','astart','aend','alabel','aprob'])
# remove all gold_ for test

count = 0
for ind in tqdm(test_df.index): 
  original = test_df['text'][ind]
  id = test_df['id'][ind]

  prediction = predictor.predict(sentence = original.replace('\"',''))

  text, spans, deps, verbs = extract_dependents(prediction) # not passed as argument above
  base = record(text)        # text re-generated from prediction includes extra spaces !!!
  sentence = normalize(text) # remove all extra spaces 
  removed = contract(sentence, base) # keep removed 

  print(id, original)
  astart=-1
  aend = -1
  cstart=-1
  cend = -1
 
  cid, clab, cstart, cend, cprob = analyze(id, c_df, rc_df, original, sentence, verbs) # consequent only !!
  print(cprob,'--->',original[cstart:cend+1],cstart,cend)
  aid, alab, astart, aend, aprob = analyze(id, a_df, ra_df, original, sentence, verbs, cstart, cend) # both
  print(aprob,'--<-',original[astart:aend+1],astart, aend) 

  if (cprob == 100 or cprob == 75 or cprob == 50 or cprob == 25) and (aprob == 100 or aprob == 75 or aprob == 50 or aprob == 25):
    cid, clab, cstart, cend, cprob = analyze(id, c_df, rc_df, original, sentence, verbs, astart, aend, False)
    print(cprob,'--->',original[cstart:cend+1],cstart,cend)
    aid, alab, astart, aend, aprob = analyze(id, a_df, ra_df, original, sentence, verbs, cstart, cend, False)
    print(aprob,'--<-',original[astart:aend+1],astart,aend)

  if aprob=='-50':   
    cstart= astart   
    cend = aend     
    aid, alab, astart, aend, aprob = analyze(id, a_df, ra_df, original, sentence, verbs, cstart,cend, False)
    print(aprob,'--<-',original[astart:aend+1],astart,aend)  
  elif aprob=='-75':  
    aid, alab, astart, aend, aprob = analyze(id, a_df, ra_df, original, sentence, verbs, cstart, cend, False,True) # override
    print(aprob,'--<-',original[astart:aend+1],astart,aend) 
  elif aprob=='-100':
    cid,clab, cstart, cend, cprob = analyze(id, c_df, rc_df, original, sentence, verbs, astart, aend, False)
    print(cprob,'--->',original[cstart:cend+1],cstart,cend)

  if cprob == '-25':
    astart=cstart
    aend = cend
    cid, clab, cstart, cend, cprob = analyze(id, c_df, rc_df, original, sentence, verbs, astart, aend, True, True) # override
    print(cprob,'--->',original[cstart:cend+1],cstart,cend)
    if cprob == 0:  
      aid, alab, astart, aend, aprob = analyze(id, a_df, ra_df, original, sentence, verbs, cstart,cend,False,True) # override
      print(aprob,'--<-',original[astart:aend+1],astart,aend)

  if aprob == 50:  
    cid, clab, cstart, cend, cprob = analyze(id, c_df, rc_df, original, sentence, verbs, astart, aend)
    print(cprob,'--->',original[cstart:cend+1],cstart,cend)
    if cprob != 100 and cprob != 75 and cprob != '-75':
      aid, alab, astart, aend, aprob = analyze(id, a_df, ra_df, original, sentence, verbs, cstart, cend)
      print(aprob,'--<-',original[astart:aend+1],astart, aend)    
 
  if aprob == '-25':  
    cstart= astart    
    cend = aend      
    aid, alab, astart, aend, aprob = analyze(id, a_df, ra_df, original, sentence, verbs, cstart, cend, True, True) # override
    print(aprob,'--<-',original[astart:aend+1],astart,aend)  
    if aprob == 0:
      cid, clab, cstart, cend, cprob = analyze(id, c_df, rc_df, original, sentence, verbs, astart,aend,False,True) # override
      print(cprob,'--->',original[cstart:cend+1],cstart,cend)
  
  if aprob=='-50':   
    cstart= astart   
    cend = aend     
    aid, alab, astart, aend, aprob = analyze(id, a_df, ra_df, original, sentence, verbs, cstart,cend, False)
    print(aprob,'--<-',original[astart:aend+1],astart,aend)


  if aprob == '-25':  
    cid, clab, cstart, cend, cprob = analyze(id, c_df, rc_df, original, sentence, verbs, astart, aend)
    print(cprob,'--->',original[cstart:cend+1],cstart,cend)

  if cprob == '-25':
    aid, alab, astart, aend, aprob = analyze(id, a_df, ra_df, original, sentence, verbs, cstart, cend)
    print(aprob,'--<-',original[astart:aend+1],astart,aend)

  if aprob == '-25':  
    cid, clab, cstart, cend, cprob = analyze(id, c_df, rc_df, original, sentence, verbs, astart, aend)
    print(cprob,'--->',original[cstart:cend+1],cstart,cend)


  consequent = original[cstart:cend+1]
  antecedent = original[astart:aend+1] 

  cdep = '' # Pass
  if consequent == antecedent:
    consequent = ''
    cstart=-1
    cend = -1
    cdep = '='

  if len(consequent) > 0:
    if consequent in antecedent:
      consequent = ''
      cstart=-1
      cend = -1
      cdep = '<'
    elif antecedent in consequent:
      antecedent = consequent
      astart=cstart
      aend = cend
      aprob= cprob
      consequent = ''
      cstart=-1
      cend = -1
      cdep = '>'
    
  if len(antecedent) > 0:       
    if len(consequent) > 0:
      text = antecedent + ' | ' + consequent
    else:
      text = antecedent
  else:
    cdep = '!'      

  if len(cdep) > 0:
    count=count+1
    
  report_row = pd.Series( data={
    'sentenceID': id,
    'clabel': clab,
    'cprob': cprob,
    'cdep' : cdep,
    'consequent': consequent, # comment all gold_... 
    #'gold_consequent': test_df['consequent'][ind],
    'cstart': cstart,  
    #'gold_cstart': test_df['cstart'][ind],       
    'cend': cend,
    #'gold_cend': test_df['cend'][ind],
    #'gold_antecedent': test_df['antecedent'][ind],
    'antecedent': antecedent, 
    'astart': astart,
    #'gold_astart': test_df['astart'][ind], 
    'aend': aend,
    #'gold_aend': test_df['aend'][ind],
    'alabel': alab,
    'aprob': aprob
  }, name = str(id) )

  result_row = pd.Series( data={
    'sentenceID': id,
    'antecedent_startid':astart,
    'antecedent_endid': aend,
    'consequent_startid': cstart,
    'consequent_endid': cend
  }, name = str(id) )

  submit_df = submit_df.append(result_row, ignore_index=True)
  report_df = report_df.append(report_row, ignore_index=True)

  print(text, astart, aend, cstart, cend)
  print(cdep,'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
  sleep(1)

submit_df.to_csv(prefix+'subtask2.csv',index=False, header=['sentenceID','antecedent_startid','antecedent_endid','consequent_startid','consequent_endid']) 
report_df.to_csv(prefix+'report.csv',index=False, header=['sentenceID','clabel','cprob','cdep','consequent','cstart','cend','antecedent','astart','aend','alabel','aprob'])

In [ ]:
!zip submission_task2.zip subtask2.csv
!mv /content/submission_task2.zip /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/
!mv /content/report.csv /content/gdrive/My\ Drive/Colab\ Notebooks/Subtask-2/

In [ ]:
print('Adjusted =',count)